In [4]:
%pip install pandas as pd

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

In [10]:
%pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 1.0 MB/s eta 0:00:00a 0:00:01
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [98]:
# Define la ruta al archivo
file_path = "./libros/pedido.xlsx"

# Lee el archivo de Excel en un DataFrame
df = pd.read_excel(file_path)

In [99]:
# Define un diccionario con los nombres de las columnas actuales y los nuevos nombres
column_names = {
    'cliente1': {
        'ID GUIA': 'id_guia',
        'DESTINATARIO': 'nombre',
        'CIUDAD': 'ciudad',
        'DIRECCION DESTINATARIO': 'direccion',
        'TELEFONO': 'telefono',
        'FECHA': 'f_emi',
        'FORMA DE PAGO': 'forma_de_pago',
        'RECAUDO': 'recaudo',
        'CONTENIDO': 'contenido'
    },
    'cliente2': {
        'ID GUIA': 'id_guia_2',
        'DESTINATARIO': 'nombre_2',
        'CIUDAD': 'ciudad_2',
        'DIRECCION DESTINATARIO': 'direccion_2',
        'TELEFONO': 'telefono_2',
        'FECHA': 'f_emi_2',
        'FORMA DE PAGO': 'forma_de_pago_2',
        'RECAUDO': 'recaudo_2',
        'CONTENIDO': 'contenido_2'
    }
    # Agrega más clientes según sea necesario
}


In [100]:
# Renombra las columnas
df = df.rename(columns=column_names['cliente1'])


In [101]:
print(df.head())

   id_guia EMPRESA      f_emi                     nombre       ciudad  \
0   425255   ZAMIA 2024-04-13               Milena Perez  Bogotá D.C.   
1   425258   ZAMIA 2024-04-13           Anguely Bernal 2  Bogotá D.C.   
2   425261   ZAMIA 2024-04-13  Natalia Andrea Diaz Catro  Bogotá D.C.   
3   425262   ZAMIA 2024-04-13         Elizabeth Garreta2  Bogotá D.C.   
4   425264   ZAMIA 2024-04-13         DIANA TORRES NINCO  Bogotá D.C.   

                                           direccion    telefono  \
0          Cra 87# 18-53 torre 3 apto 104 Hayuelos -  3104048275   
1                     Cra 29 #75 a 26 santa Sofía, -  3134088908   
2        av cll 3 # 31b-66 piso 4 veraguas central -  3046438960   
3  Calle 10b No 88a-17 Reserva de San Agustín II ...  3132685210   
4             carrera 78i numero 51a23sur Catalina -  3123832144   

  forma_de_pago      recaudo  \
0      EFECTIVO   $69.900,00   
1      EFECTIVO   $97.000,00   
2      EFECTIVO   $69.900,00   
3      EFECTIVO   $69.90

In [54]:
from sqlalchemy import create_engine, Table, MetaData, select, func
from sqlalchemy.exc import NoSuchTableError

In [102]:
from sqlalchemy import create_engine, Table, MetaData, select, func
from sqlalchemy.exc import NoSuchTableError

# Crea una conexión a la base de datos
engine = create_engine('sqlite:///my_database.db')  # Reemplaza esto con la cadena de conexión a tu base de datos

# Define la tabla
metadata = MetaData()
try:
    ordenes = Table('ordenes', metadata, autoload_with=engine)

    # Haz una consulta para obtener el valor máximo de 'id'
    with engine.connect() as connection:
        result = connection.execute(select([func.max(ordenes.c.id)]))
        max_id = result.scalar()

    # Añade 7000000000 al valor máximo de 'id'
    serial = max_id + 7000000000

    # Añade la columna 'serial' al DataFrame
    df['serial'] = serial

    df = df[['serial', 'id_guia', 'nombre', 'ciudad', 'direccion', 'telefono', 'f_emi', 'forma_de_pago', 'recaudo', 'contenido']]


    # Inserta el DataFrame en la tabla 'ordenes'
    df.to_sql('ordenes', con=engine, if_exists='append', index=False)
except NoSuchTableError:
    # Si la tabla no existe, establece 'serial' en 7000000000 más el índice del DataFrame
    df['serial'] = 7000000000 + df.index

    df = df[['serial', 'id_guia', 'nombre', 'ciudad', 'direccion', 'telefono', 'f_emi', 'forma_de_pago', 'recaudo', 'contenido']]


    # Inserta el DataFrame en la tabla 'ordenes'
    df.to_sql('ordenes', con=engine, if_exists='fail', index=False)

print(df.head())

       serial  id_guia                     nombre       ciudad  \
0  7000000000   425255               Milena Perez  Bogotá D.C.   
1  7000000001   425258           Anguely Bernal 2  Bogotá D.C.   
2  7000000002   425261  Natalia Andrea Diaz Catro  Bogotá D.C.   
3  7000000003   425262         Elizabeth Garreta2  Bogotá D.C.   
4  7000000004   425264         DIANA TORRES NINCO  Bogotá D.C.   

                                           direccion    telefono      f_emi  \
0          Cra 87# 18-53 torre 3 apto 104 Hayuelos -  3104048275 2024-04-13   
1                     Cra 29 #75 a 26 santa Sofía, -  3134088908 2024-04-13   
2        av cll 3 # 31b-66 piso 4 veraguas central -  3046438960 2024-04-13   
3  Calle 10b No 88a-17 Reserva de San Agustín II ...  3132685210 2024-04-13   
4             carrera 78i numero 51a23sur Catalina -  3123832144 2024-04-13   

  forma_de_pago      recaudo  \
0      EFECTIVO   $69.900,00   
1      EFECTIVO   $97.000,00   
2      EFECTIVO   $69.900,00   


In [103]:
print(df.head())

       serial  id_guia                     nombre       ciudad  \
0  7000000000   425255               Milena Perez  Bogotá D.C.   
1  7000000001   425258           Anguely Bernal 2  Bogotá D.C.   
2  7000000002   425261  Natalia Andrea Diaz Catro  Bogotá D.C.   
3  7000000003   425262         Elizabeth Garreta2  Bogotá D.C.   
4  7000000004   425264         DIANA TORRES NINCO  Bogotá D.C.   

                                           direccion    telefono      f_emi  \
0          Cra 87# 18-53 torre 3 apto 104 Hayuelos -  3104048275 2024-04-13   
1                     Cra 29 #75 a 26 santa Sofía, -  3134088908 2024-04-13   
2        av cll 3 # 31b-66 piso 4 veraguas central -  3046438960 2024-04-13   
3  Calle 10b No 88a-17 Reserva de San Agustín II ...  3132685210 2024-04-13   
4             carrera 78i numero 51a23sur Catalina -  3123832144 2024-04-13   

  forma_de_pago      recaudo  \
0      EFECTIVO   $69.900,00   
1      EFECTIVO   $97.000,00   
2      EFECTIVO   $69.900,00   


In [111]:
%pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [112]:
import re
import unidecode

def expandir_contenido(df):
    # Extrae todas las secuencias de 'cantidad * producto.' de 'contenido'
    matches = re.findall(r'(\d+) \* ([^.]+)\.', df['contenido'])

    # Crea una nueva fila por cada secuencia de 'cantidad * producto.'
    nuevas_filas = []
    for cantidad, producto in matches:
        # Si aparece la palabra 'Dúo', multiplica la cantidad por dos y borra la palabra 'Dúo'
        if 'Dúo' in producto:
            cantidad = str(int(cantidad) * 2)
            producto = producto.replace('Dúo', '').strip()

        # Si aparece la palabra 'Crema', borra la palabra 'Crema'
        if 'Crema' in producto:
            producto = producto.replace('Crema', '').strip()

        nuevas_filas.append({'serial': df['serial'], 'cantidad': cantidad, 'producto': producto})

    return nuevas_filas

In [113]:
nuevas_filas = []
for _, row in df.iterrows():
    nuevas_filas.extend(expandir_contenido(row))
df_inventario = pd.DataFrame(nuevas_filas)
df_inventario['producto'] = df_inventario['producto'].str.lower().apply(unidecode.unidecode)

print(df_inventario)

        serial cantidad   producto
0   7000000000        1  aclarante
1   7000000001        2  aclarante
2   7000000002        1  aclarante
3   7000000003        1  aclarante
4   7000000004        1   antiacne
..         ...      ...        ...
86  7000000066        1  aclarante
87  7000000067        1   antiacne
88  7000000067        1  aclarante
89  7000000067        1    slimgel
90  7000000068        1  aclarante

[91 rows x 3 columns]


In [114]:
print(df_inventario.tail(50))

        serial cantidad   producto
41  7000000030        1  aclarante
42  7000000031        2  aclarante
43  7000000032        1  aclarante
44  7000000033        1  aclarante
45  7000000034        1   antiacne
46  7000000034        1    slimgel
47  7000000035        1  aclarante
48  7000000036        1  aclarante
49  7000000037        2   antiacne
50  7000000037        1    slimgel
51  7000000038        2  aclarante
52  7000000039        2   antiacne
53  7000000040        1  aclarante
54  7000000041        2  aclarante
55  7000000042        2  aclarante
56  7000000043        1   antiacne
57  7000000044        2  aclarante
58  7000000045        1   antiacne
59  7000000045        1  aclarante
60  7000000046        1   antiacne
61  7000000046        1  aclarante
62  7000000047        1   antiacne
63  7000000048        1  aclarante
64  7000000049        2  aclarante
65  7000000049        1    slimgel
66  7000000050        1  aclarante
67  7000000051        1  aclarante
68  7000000052      